In [ ]:
!pip install tensorboardx
!mkdir checkpoints
!mkdir results


     |████████████████████████████████| 204kB 6.5MB/s 


In [ ]:
!cp /content/drive/"My Drive"/correctedmetaclassifiermnist_checkpoint.zip ./
!unzip ./correctedmetaclassifiermnist_checkpoint.zip

Archive:  ./correctedmetaclassifiermnist_checkpoint.zip
  inflating: checkpoints/run4_gan_discriminator_0  
  inflating: checkpoints/run4_gan_discriminator_1000  
  inflating: checkpoints/run4_gan_discriminator_10000  
  inflating: checkpoints/run4_gan_discriminator_11000  
  inflating: checkpoints/run4_gan_discriminator_12000  
  inflating: checkpoints/run4_gan_discriminator_13000  
  inflating: checkpoints/run4_gan_discriminator_14000  
  inflating: checkpoints/run4_gan_discriminator_15000  
  inflating: checkpoints/run4_gan_discriminator_16000  
  inflating: checkpoints/run4_gan_discriminator_17000  
  inflating: checkpoints/run4_gan_discriminator_18296  
  inflating: checkpoints/run4_gan_discriminator_19296  
  inflating: checkpoints/run4_gan_discriminator_2000  
  inflating: checkpoints/run4_gan_discriminator_20296  
  inflating: checkpoints/run4_gan_discriminator_21296  
  inflating: checkpoints/run4_gan_discriminator_22296  
  inflating: checkpoints/run4_gan_discriminator_23296 

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
import torchvision
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data.sampler import BatchSampler
from itertools import cycle
from tensorboardX import SummaryWriter
import torch.autograd as autograd
from tqdm import tqdm
import matplotlib.pyplot as plt


In [ ]:

class BalancedBatchSampler(BatchSampler):
    """
    BatchSampler - from a MNIST-like dataset, samples n_classes and within these classes samples n_samples.
    Returns batches of size n_classes * n_samples
    """

    def __init__(self, dataset, n_classes, n_samples,custom_classes =None):
        loader = DataLoader(dataset)
        self.labels_list = []
        self.custom_classes = custom_classes
        for _, label in loader:
            self.labels_list.append(label)
        self.labels = torch.LongTensor(self.labels_list)
        self.labels_set = list(set(self.labels.numpy()))
        self.label_to_indices = {label: np.where(self.labels.numpy() == label)[0]
                                 for label in self.labels_set}
        for l in self.labels_set:
            np.random.shuffle(self.label_to_indices[l])
        self.used_label_indices_count = {label: 0 for label in self.labels_set}

        self.count = 0
        self.n_classes = n_classes
        self.n_samples = n_samples
        self.dataset = dataset
        self.batch_size = self.n_samples * self.n_classes

    def __iter__(self):
        self.count = 0
        while self.count + self.batch_size < len(self.dataset):
            classes = np.random.choice(self.labels_set, self.n_classes, replace=False)
            indices = []
            if self.custom_classes!=None:
                classes = self.custom_classes
            for class_ in classes:
                indices.extend(self.label_to_indices[class_][
                               self.used_label_indices_count[class_]:self.used_label_indices_count[
                                                                         class_] + self.n_samples])
                self.used_label_indices_count[class_] += self.n_samples
                if self.used_label_indices_count[class_] + self.n_samples > len(self.label_to_indices[class_]):
                    np.random.shuffle(self.label_to_indices[class_])
                    self.used_label_indices_count[class_] = 0
            yield indices
            self.count += self.n_classes * self.n_samples

    def __len__(self):
        return len(self.dataset) // self.batch_size

In [ ]:


writer = SummaryWriter('runs_gan/wgangp_mnist')

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


img_size = 28
##Simulataneous training of different classes
n_classes = 1
#custom_classes = np.random.randint(0,10,n_classes)
custom_classes = np.array([9])
#custom_classes = None
class_batch = 64
transform = transforms.Compose([torchvision.transforms.Resize(img_size),torchvision.transforms.ToTensor()])
dataset = torchvision.datasets.MNIST(root="./", train=True,download=True,transform=transform)
#balanced_batch_sampler = BalancedBatchSampler(dataset, n_classes, class_batch,custom_classes)
batch_size = 64

#dataloader = torch.utils.data.DataLoader(dataset, batch_sampler=balanced_batch_sampler)
class_dataset = torch.utils.data.dataset.Subset(dataset, np.where(dataset.targets==9)[0])
dataloader = torch.utils.data.DataLoader(class_dataset,batch_size=batch_size,shuffle=True,num_workers=4)
##parameters

dim = 64
LAMBDA = 10
epochs= 2000
n_z = 128
k = 10

Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw


Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw



Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw


Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw
Processing...
Done!


/pytorch/torch/csrc/utils/tensor_numpy.cpp:141: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program.


In [ ]:


class netD(nn.Module):
    def __init__(self):
        super(netD,self).__init__()
        self.conv1 = nn.Conv2d(1,dim,5,stride=2,padding=2)
        self.act1 = nn.ReLU(True)
        self.conv2 = nn.Conv2d(dim,dim*2,5,stride=2,padding=2)
        self.act2 = nn.ReLU(True)
        self.conv3 = nn.Conv2d(dim*2,dim*4,5,stride=2,padding=2)
        self.act3 = nn.ReLU(True)
        self.fc1 = nn.Linear(4*4*4*dim,1)

    def forward(self,input):
        input = input.view(-1,1,28,28)

        l1_1 = self.conv1(input)
        l1_2 = self.act1(l1_1)
        l2_1 = self.conv2(l1_2)
        l2_2 = self.act2(l2_1)
        l3_1 = self.conv3(l2_2)
        l3_2 = self.act3(l3_1)

        l4_1 = self.fc1(l3_2.view(-1,4*4*4*dim))
        return l4_1.view(-1)


class netG(nn.Module):
    def __init__(self):
        super(netG,self).__init__()
        self.convt1 = nn.Linear(128,4*4*4*dim)
        self.act1 = nn.ReLU(True)
        self.convt2 = nn.ConvTranspose2d(4*dim,dim*2,5)
        self.act2 = nn.ReLU(True)
        self.convt3 = nn.ConvTranspose2d(dim*2,dim,5)
        self.act3 = nn.ReLU(True)
        self.convt4 = nn.ConvTranspose2d(dim,1,8,stride=2)
        self.sigmoid = nn.Sigmoid()

    def forward(self,input):
        #input = input.view(-1,1,28,28)
        l1_1 = self.convt1(input)
        l1_2 = self.act1(l1_1)
        l1_2 = l1_2.view(-1,4*dim,4,4)
        l2_1 = self.convt2(l1_2)
        l2_2 = self.act2(l2_1)
        l2_2 = l2_2[:,:,:7,:7]
        l3_1 = self.convt3(l2_2)
        l3_2 = self.act3(l3_1)
        l4_1 = self.convt4(l3_2)
        output = self.sigmoid(l4_1)
        return output.view(-1,img_size*img_size)


In [ ]:

fixed_batch = torch.rand(class_batch,128,device=device,requires_grad=False)

def calc_gradient_penalty(netD, real_data, fake_data):
    #print real_data.size()
    alpha = torch.rand(class_batch, 1).to(device)
    alpha = alpha.expand(real_data.size())
    interpolates = alpha * real_data + ((1 - alpha) * fake_data)

    interpolates = autograd.Variable(interpolates, requires_grad=True)

    disc_interpolates = netD(interpolates)

    gradients = autograd.grad(outputs=disc_interpolates, inputs=interpolates,
                              grad_outputs=torch.ones(disc_interpolates.size()).to(device),
                              create_graph=True, retain_graph=True, only_inputs=True)[0]

    gradient_penalty = ((gradients.norm(2, dim=1) - 1) ** 2).mean() * LAMBDA
    return gradient_penalty
#criterion = nn.BCEWithLogitsLoss()
#metanet = netD().to(device)
#tasknet = netD().to(device)
#taskoptim = optim.Adam(tasknet.parameters(),lr=0.0001)
#metaoptim = optim.Adam(metanet.parameters(),lr=0.00001)

##training

generator = netG().to(device)
discriminator = netD().to(device)
optimizerD = optim.Adam(discriminator.parameters(), lr=1e-4, betas=(0.5, 0.9))
optimizerG = optim.Adam(generator.parameters(), lr=1e-4, betas=(0.5, 0.9))


In [ ]:
discriminator.load_state_dict(torch.load("./checkpoints/run4_gan_discriminator_27296"))

<All keys matched successfully>

In [ ]:

iters=0
for epoch in range(epochs):
  data_iterator = iter(dataloader)
  index = 0
  #print("coca")
  while index+k<=dataloader.__len__():
    for p in discriminator.parameters():
      p.requires_grad=True
    for _ in range(k):
      #print("cola")
      discriminator.zero_grad()
      x_real = data_iterator.next()[0].to(device)
      x_real = x_real.view(-1,img_size*img_size)
      b_size = x_real.shape[0]
      output = discriminator(x_real).view(-1)
      D_real = output.mean()

      noise = torch.randn(b_size,n_z,device= device)
      fake_data = generator(noise)
      output = discriminator(fake_data.detach()).view(-1)
      D_fake = output.mean()
      D_loss = D_fake - D_real
      D_loss.backward()
      gradient_p = calc_gradient_penalty(discriminator,x_real.detach(),fake_data.detach())
      gradient_p.backward()
      total_D_loss = D_loss + gradient_p
      optimizerD.step()
    index = index + k
    for p in discriminator.parameters():
      p.requires_grad = False
    generator.zero_grad()
    noise = torch.randn(b_size,n_z,device=device)
    fake_data = generator(noise)
    output = discriminator(fake_data).view(-1)
    G_loss = -output.mean()
    G_loss.backward()
    optimizerG.step()
    writer.add_scalar('Discriminator_Loss', total_D_loss,iters)
    writer.add_scalar('Generator_Loss',G_loss,iters)
    if iters%100==0:
      print("Epoch : "+str(epoch)+" D_Loss: " + str(total_D_loss.item())+"  G_loss: " +str(G_loss.item()))
    if epoch%50==0:
      
      torch.save(generator.state_dict(),'./checkpoints/metagan_generator_'+str(iters))
      torch.save(discriminator.state_dict(),'./checkpoints/metagan_discriminator_'+str(iters))
      with torch.no_grad():
        sample_batch = generator(fixed_batch).detach().cpu()
        sample_batch = sample_batch.view(-1,1,28,28)
        grid = np.concatenate([np.concatenate([sample_batch[i * 4 + j] for j in range(3)], axis=-2) for i in range(int(class_batch/4))], axis=-1)
        sample_batch = sample_batch.view(-1,28,28)
        grid2 = np.concatenate([np.concatenate([sample_batch[i * 4 + j] for j in range(3)], axis=-2) for i in range(int(class_batch/4))], axis=-1)
        plt.imsave("./results/"+str(epoch)+"_"+str(iters)+".png",grid2)
        writer.add_image('sample_output',grid,iters)
    iters = iters+1






 






Epoch : 0 D_Loss: -2.392205238342285  G_loss: 2.2760512828826904
Epoch : 11 D_Loss: -6.311069011688232  G_loss: -1.3988926410675049
Epoch : 22 D_Loss: -4.738125324249268  G_loss: -0.43008947372436523
Epoch : 33 D_Loss: -4.284755706787109  G_loss: -1.0587581396102905
Epoch : 44 D_Loss: -3.7300870418548584  G_loss: -0.5803325772285461
Epoch : 55 D_Loss: -3.0824708938598633  G_loss: -1.6112427711486816
Epoch : 66 D_Loss: -2.881756067276001  G_loss: -0.5878746509552002
Epoch : 77 D_Loss: -3.2487080097198486  G_loss: -1.0606536865234375
Epoch : 88 D_Loss: -2.582897424697876  G_loss: -1.3888922929763794
Epoch : 100 D_Loss: -2.613327741622925  G_loss: -1.1643283367156982
Epoch : 111 D_Loss: -2.354588270187378  G_loss: -1.1145910024642944
Epoch : 122 D_Loss: -2.394170045852661  G_loss: 0.14611536264419556
Epoch : 133 D_Loss: -2.450990915298462  G_loss: -0.6327723264694214
Epoch : 144 D_Loss: -2.496802806854248  G_loss: -0.4549523591995239
Epoch : 155 D_Loss: -2.2458291053771973  G_loss: -0.364

In [ ]:
!zip -r corrmetawgan_mnist_digit9_m10_result.zip ./results/
!zip -r corrmetawgan_mnist_digit9_m10_checkpoint.zip ./checkpoints/
!zip -r corrmetawgan_mnist_digit9_m10_log.zip ./runs_gan

In [ ]:
#!rm -r ./checkpoints/*
#!rm -r ./results/*
#!rm -r ./runs_gan/*

In [ ]:
!mv corrmetawgan_mnist_digit9_m10_result.zip /content/drive/"My Drive"/
!mv corretawgan_mnist_digit9_m10_checkpoint.zip /content/drive/"My Drive"/
!mv corrmetawgan_mnist_digit9_m10_log.zip /content/drive/"My Drive"/



In [ ]:
!nvidia-smi

In [ ]:
!ls ./checkpoints

run4_gan_discriminator_0      run4_gan_discriminator_21296
run4_gan_discriminator_1000   run4_gan_discriminator_22296
run4_gan_discriminator_10000  run4_gan_discriminator_23296
run4_gan_discriminator_11000  run4_gan_discriminator_24296
run4_gan_discriminator_12000  run4_gan_discriminator_25296
run4_gan_discriminator_13000  run4_gan_discriminator_26296
run4_gan_discriminator_14000  run4_gan_discriminator_27296
run4_gan_discriminator_15000  run4_gan_discriminator_3000
run4_gan_discriminator_16000  run4_gan_discriminator_4000
run4_gan_discriminator_17000  run4_gan_discriminator_5000
run4_gan_discriminator_18296  run4_gan_discriminator_6000
run4_gan_discriminator_19296  run4_gan_discriminator_7000
run4_gan_discriminator_2000   run4_gan_discriminator_8000
run4_gan_discriminator_20296  run4_gan_discriminator_9000
